# Reed-Solomon code characteristic table
This is the local version of the notebook for generating a table of Reed-Solomon code parameters. To run the google colab version, click [here](https://colab.research.google.com/github/CaiGroup/UntanglingBarcodes/blob/master/codebook_generation/get_RS_codebooks/colab/Make_RS_Code_Table.jl.ipynb).

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `D:\SeqFISH+Processing_project\UntanglingBarcodes\codebook_generation\get_RS_codebooks\local`


In [2]:
using DataFrames
using CSV

In [3]:
function get_num_codewords_of_weight(q,w,d)
    i = collect(0:(w-d))
    (q-1)*binomial(q-1,w)*sum(((-1) .^ i) .* (binomial.(w-1, i)) .* (q .^ ((w - d) .- i)))
end

function get_num_codewords_of_weight_vn(q,n,w,d)
    i = collect(0:(w-d))
    (q-1)*binomial(n,w)*sum(((-1) .^ i) .* (binomial.(w-1, i)) .* (q .^ ((w - d) .- i)))
end

get_num_codewords_of_weight_vn (generic function with 1 method)

In [4]:
qs = [5,7,8,9,11,13,16,17]
params = []
ncws = []
qcs = []
mds = []
nimg = []
ws = []
ns = []
for q in qs
    for md in 2:5
        for nmod in [-3, -2, -1, 0, 1, 2]
            push!(ncws, get_num_codewords_of_weight_vn.(q,q-1+nmod,3:6,md))
            nimgs = (q-1) * (q-1+nmod)
            push!(qcs, q)
            push!(ns, q-1+nmod)
            push!(mds, md)
            push!(nimg, nimgs)
        end
    end
end
            

In [5]:
code_table = DataFrame(hcat(ncws...)', "w" .* string.(3:6));


In [6]:
code_table = DataFrame(hcat(ncws...)', "w" .* string.(3:6))
#code_table = 
insertcols!(code_table,1, ("mhd" => mds))

insertcols!(code_table,1, ("n" => ns))
#code_table = 
insertcols!(code_table,1, ("q" => qcs))
insertcols!(code_table,1, ("nImages" => nimg))

generating_notebook = []

for r in eachrow(code_table)
    if r.q in [8, 9]
        push!(generating_notebook, "gen_extended_codes_ext_fields_q9_n9-10_nmk4.ipynb")
    elseif r.n == r.q - 1
        push!(generating_notebook, "gen_RS_q11k7_code.jl.ipynb")
    elseif r.n >= r.q
        push!(generating_notebook, "gen_extended_codes_cuda.jl.ipynb")
    elseif  r.n < r.q - 1
        push!(generating_notebook, "gen_RS_q11k7_code.jl.ipynb -> get_shortened_cb.jl.ipynb")
    end
end

code_table[!, "Generating Notebook"] = generating_notebook;


In [7]:
code_table

Row,nImages,q,n,mhd,w3,w4,w5,w6,Generating Notebook
,Any,Any,Any,Any,Int64,Int64,Int64,Int64,Any
1,4,5,1,2,0,0,0,0,gen_RS_q11k7_code.jl.ipynb -> get_shortened_cb.jl.ipynb
2,8,5,2,2,0,0,0,0,gen_RS_q11k7_code.jl.ipynb -> get_shortened_cb.jl.ipynb
3,12,5,3,2,12,0,0,0,gen_RS_q11k7_code.jl.ipynb -> get_shortened_cb.jl.ipynb
4,16,5,4,2,48,52,0,0,gen_RS_q11k7_code.jl.ipynb
5,20,5,5,2,120,260,204,0,gen_extended_codes_cuda.jl.ipynb
6,24,5,6,2,240,780,1224,820,gen_extended_codes_cuda.jl.ipynb
7,4,5,1,3,0,0,0,0,gen_RS_q11k7_code.jl.ipynb -> get_shortened_cb.jl.ipynb
8,8,5,2,3,0,0,0,0,gen_RS_q11k7_code.jl.ipynb -> get_shortened_cb.jl.ipynb
9,12,5,3,3,4,0,0,0,gen_RS_q11k7_code.jl.ipynb -> get_shortened_cb.jl.ipynb


In [8]:
CSV.write("output/RS_code_spec_table.csv", code_table)

"output/RS_code_spec_table.csv"